<a href="https://colab.research.google.com/github/kavyajeetbora/geemap/blob/master/end_to_end_earth_engine/Module_01_Earth_Engine_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Working with Image Collections

An ImageCollection is a dataset that consists of images takes at different time and locations - usually from the same satellite or data provider.

You can load a collection by searching the Earth Engine Data Catalog for the `ImageCollection ID`. Search for the`Sentinel-2 Level 1C` dataset and you will find its id `COPERNICUS/S2_SR`

**About Sentinel-2 Satellite**

![](https://content.satimagingcorp.com/media2/filer_public_thumbnails/filer_public/b8/3b/b83b4782-bd4f-404f-badf-0bc160cdd959/cms_page_media1530sentinel-2.jpeg__400.0x305.0_q85_subsampling-2.jpg)

The Copernicus Program is an ambitious initiative headed by the European Commission in partnership with the European Space Agency (ESA). The Sentinels are a constellation of satellites developed by ESA to operationalize the Copernicus program, which include all-weather radar images from Sentinel-1A and 1B, high-resolution optical images from Sentinel-2A and 2B, ocean and land data suitable for environmental and climate monitoring from Sentinel-3, as well as air quality data from Sentinel-5P.

Sentinel-2 is a wide-swath, high-resolution, multi-spectral imaging mission supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas.

The Sentinel-2 data contain 13 UINT16 spectral bands representing TOA reflectance scaled by 10000

**What is cloud masking ?**

Cloud masking on GEE using Landsat 8 involves the identification and removal of cloud-contaminated pixels from satellite imagery.

'QA60' band refers to the Quality Assessment band of Sentinel-2 images, which contains information about various atmospheric conditions, including cloud and cirrus cover. This band is used to identify and mask out pixels affected by clouds and cirrus in the image



In [19]:
def maskS2clouds(image):
    '''
    Function to mask clouds using the Sentinel-2 QA band
    @param {ee.Image} image Sentinel-2 image
    @return {ee.Image} cloud masked Sentinel-2 image
    '''
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

dataset = ee.ImageCollection('COPERNICUS/S2').filterDate('2018-01-01', '2018-06-30') \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds)

Map = geemap.Map(center=(12.9407, 77.5925), zoom=12)

viz_params = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2']
}
Map.addLayer(dataset.median(), viz_params, 'RGB')

Map

Map(center=[12.9407, 77.5925], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

## Exercise

Find the 'Sentinel-2 Level-1C' dataset page
https://developers.google.com/earth-engine/datasets

Copy/paste the code snippet

Change the code to display images for your home city

In [8]:
Map = geemap.Map(center=(26.6401,92.7964), zoom=13)

Map.addLayer(dataset, viz_params, name='SENTINEL')
Map

Map(center=[26.6401, 92.7964], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

## Filtering Image Collection

- The collection contains all imagery ever collected by the sensor. The entire collections are not very useful.
- Most applications require a subset of the images. We use filters to select the appropriate images.
-There are many types of filter functions, look at ee.Filter... module to see all available filters. Select a filter and then run the filter() function with the filter parameters

We will learn about 3 main types of filtering techniques

1. **Filter by metadata**: You can apply a filter on the image metadata using filters such as ee.Filter.eq(), ee.Filter.lt() etc. You can filter by PATH/ROW values, Orbit number, Cloud cover etc.
2. **Filter by date**: You can select images in a particular date range using filters such as ee.Filter.date().
3. **Filter by location**: You can select the subset of images with a bounding box, location or geometry using the ee.Filter.bounds(). You can also use the drawing tools to draw a geometry for filtering.

In [21]:
center = [77.60412933051538, 12.952912912328241]
Map = geemap.Map(center=center, zoom=12)
geometry = ee.Geometry.Point(center)
Map.centerObject(geometry, zoom=10)

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

## Filter by metadata
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

## Filter by date
filtered = s2.filter(ee.Filter.date('2019-01-01', '2020-01-01'))

## Filter by location
filtered = s2.filter(ee.Filter.bounds(geometry))


filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.bounds(geometry))

Map.addLayer(filtered, viz_params, name="SENTINEL")

print(filtered.size())

Map

ee.Number({
  "functionInvocationValue": {
    "functionName": "Collection.size",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "ImageCollection.load",
                            "arguments": {
                              "id": {
                                "constantValue": "COPERNICUS/S2_HARMONIZED"
                              }
                            }
                          }
                        

Map(center=[12.952912912328241, 77.60412933051538], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
help(ee.filter)

Help on module ee.filter in ee:

NAME
    ee.filter - Collection filters.

DESCRIPTION
    Example usage:
      Filter('time', low, high)
         .bounds(ring)
         .eq('time', value)
         .lt('time', value)

CLASSES
    ee.computedobject.ComputedObject(ee.encodable.Encodable)
        Filter
    
    class Filter(ee.computedobject.ComputedObject)
     |  Filter(*args, **kwargs)
     |  
     |  An object to represent collection filters.
     |  
     |  Method resolution order:
     |      Filter
     |      ee.computedobject.ComputedObject
     |      ee.encodable.Encodable
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  Not(self)
     |      Returns the opposite of this filter.
     |      
     |      Returns:
     |        The negated filter, which will match iff this filter doesn't.
     |  
     |  __init__(self, filter_=None)
     |      Construct a filter.
     |      
     |      This constructor accepts the following args:
     |  